# Dreambooth
Notebook implementation by Joe Penna (@MysteryGuitarM on Twitter) - Improvements by David Bielejeski
Personal workflow improvements by Lonnon Foster (lonnon@lonnon.com)

Latest information on:
https://github.com/JoePenna/Dreambooth-Stable-Diffusion

## Build Environment

In [ ]:
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
!pip install gdown
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
!pip install pydrive2

# Import some modules we'll need
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

## Download SD Model

In [ ]:
MODEL_ID = '1uOg7A5w_ypSxhoG176Naf0HICfKTLxP9'

gauth = GoogleAuth()
gauth.CommandLineAuth()
drive = GoogleDrive(gauth)
sd_model = drive.CreateFile({'id': MODEL_ID})
sd_model.GetContentFile('model.ckpt')
print('✅ model.ckpt successfully downloaded')

## Download Regularization Images

In [ ]:
REG_ID = '1xPee7IopXCmzyG45EccIuCSTgbjItsQP'

reg_dir = drive.CreateFile
image_list = drive.ListFile(
    {'q': f"'{REG_ID}' in parents"}
).GetList()
for image in image_list:
    ifile = drive.CreateFile({'id': image['id']})
    ifile.GetContentFile(os.path.join('regularization_images', image['title']))
print('✅ regularization images successfully downloaded')

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

In [ ]:
project_name = "project_name"
max_training_steps = 2000
token = "firstNameLastName"
class_word = "person"

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root "/workspace/Dreambooth-Stable-Diffusion/regularization_images" \
 -n "{project_name}" \
 --gpus 0, \
 --data_root "/workspace/Dreambooth-Stable-Diffusion/training_images" \
 --max_training_steps {max_training_steps} \
 --class_word "{class_word}" \
 --token "{token}" \
 --no-test

## Copy current checkpoint
Run as necessary during training to take a snapshot of the most recent checkpoint.

In [ ]:
current_steps = '1600'

directory_paths = !ls -d logs/*
last_checkpoint_file = os.path.join(directory_paths[-1], "checkpoints/last.ckpt")
new_checkpoint_file = f"{project_name}_{current_steps}"

!cp "{last_checkpoint_file}" "{new_checkpoint_file}"

## Upload checkpoint to google drive

In [ ]:
new_model = drive.CreateFile()
new_model.SetContentFile(new_checkpoint_file)
new_model.Upload()